## Pull git repo and setup environment

In [1]:
!git clone https://github.com/Hihi1310/vietnamese-interpreter.git

Cloning into 'vietnamese-interpreter'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 53 (delta 19), reused 41 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 31.13 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
%cd /content/vietnamese-interpreter
!pip -q install -r requirements.txt

/content/vietnamese-interpreter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [3]:
# Verify GPU is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU device: Tesla T4


## Get some test data from google/fleurs dataset on Hugging Face

In [5]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
import torchaudio
import os
import random

# Ensure the sample output directory exists
output_dir = "/content/sampled_audio"
os.makedirs(output_dir, exist_ok=True)

# Sample 10 random indices from the dataset and save the sampled audio records as WAV files
def fleurs_sample(data, sample_num, note):
  sample_indices = random.sample(range(len(fleurs)), sample_num)
  for i, idx in enumerate(sample_indices):
    audio_data = fleurs[idx]['audio']['array']
    sampling_rate = fleurs[idx]['audio']['sampling_rate']
    output_path = os.path.join(output_dir, f"sample_{i+1}_{note}.wav")
    torchaudio.save(output_path, torch.tensor(audio_data).unsqueeze(0), sampling_rate)

# download Vietnamese audio data from hugging face
fleurs_vn = load_dataset("google/fleurs", "vi_vn", split="test")

fleurs_sample(fleurs_vn, 10, "vn")
# download English  audio data from hugging face
fleurs = load_dataset("google/fleurs", "en_us", split="test")

fleurs_sample(fleurs_vn, 10, "en")


data/en_us/audio/train.tar.gz:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

data/en_us/audio/dev.tar.gz:   0%|          | 0.00/171M [00:00<?, ?B/s]

data/en_us/audio/test.tar.gz:   0%|          | 0.00/290M [00:00<?, ?B/s]

train.tsv: 0.00B [00:00, ?B/s]

dev.tsv: 0.00B [00:00, ?B/s]

test.tsv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Run full system pipline

after running the code for downloading and sampling data from Hugging Face you can find .WAV audio files in sample_audio folder.<br><br>
Then pick the sample you want and put it into data/input folder in the repo then take the file path and run a cmd on colab with the code bellow with the correct file path.

In [13]:
!python src/main.py -f data/input/sample_10.wav -s vi

2025-07-13 16:13:20.793669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752423200.813831   14292 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752423200.820056   14292 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-13 16:13:20.840337: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Vietnamese Interpreter POC
INFO:system:Vietnamese Interpreter system initialized
INFO:transcription:Audio transcriber initial